# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

**Reminder about our [Rules for hand-in assignments](https://www.ida.liu.se/~TDDE16/exam.en.shtml#handins) and the [Policy on cheating and plagiarism](https://www.ida.liu.se/~TDDE16/exam.en.shtml#cheating)**

We start by loading spaCy:

In [1]:
import spacy

nlp = spacy.load('en_core_web_sm')

The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [2]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(' '))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [3]:
import bz2
import csv
import pandas as pd

with bz2.open('ner-train.tsv.bz2', 'rt', encoding = 'cp850') as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

with bz2.open('ner-dev.tsv.bz2', 'rt', encoding = 'cp850') as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [4]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [5]:
import numpy as np

def evaluation_report(gold, pred):
    """Print precision, recall, and F1 score.
    
    Args:
        gold: The set with the gold-standard values.
        pred: The set with the predicted values.
    
    Returns:
        Nothing, but prints the precision, recall, and F1 values computed
        based on the specified sets.
    """
    prec = len(gold & pred)/len(pred)
    rec = len(gold & pred)/len(gold)
    f1 = 2 *(prec * rec) / (prec + rec)
    print("precision:" ,prec , ", recall: " , rec , ", f1: " , f1)
    pass

To test your code, you can run the following cell:

In [6]:
evaluation_report(set(range(3)), set(range(5)))

precision: 0.6 , recall:  1.0 , f1:  0.7499999999999999


This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [7]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    for i in df.itertuples():
        yield (i.sentence_id, i.beg, i.end)

To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [8]:
spans_dev_gold = set(gold_spans(df_dev))
print(len(spans_dev_gold))

5917


Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [9]:
df = df_dev 
data = set()
for i in df.drop_duplicates('sentence_id').itertuples():
    doc = nlp(i.sentence)
    for ent in doc.ents:
        data.add((i.sentence_id, ent.start, ent.end))

In [10]:
evaluation_report(spans_dev_gold, data)

precision: 0.5383185017930668 , recall:  0.6849754943383471 , f1:  0.6028558679161089


## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Here is a function that prints the false positives as well as the false negatives spans for a data frame, given a reference set of gold-standard spans and a candidate set of predicted spans.

In [11]:
from collections import defaultdict

def error_report(df, spans_gold, spans_pred):
    false_pos = defaultdict(list)
    for s, b, e in spans_pred - spans_gold:
        false_pos[s].append((b, e))
    false_neg = defaultdict(list)
    for s, b, e in spans_gold - spans_pred:
        false_neg[s].append((b, e))
    for row in df.drop_duplicates('sentence_id').itertuples():
        if row.sentence_id in false_pos or row.sentence_id in false_neg:
            print('Sentence:', row.sentence)
            for b, e in false_pos[row.sentence_id]:
                print('  FP:', ' '.join(row.sentence.split()[b:e]))
            for b, e in false_neg[row.sentence_id]:
                print('  FN:', ' '.join(row.sentence.split()[b:e]))

Use this function to inspect and analyse the errors that the automated prediction makes. Can you see any patterns? Base your analysis on the first 500 rows of the training data. Summarize your observations in a short text.

In [12]:
df = df_dev[:500]
spans_dev_gold = set(gold_spans(df))
data = set()
for i in df.drop_duplicates('sentence_id').itertuples():
    doc = nlp(i.sentence)
    for ent in doc.ents:
        data.add((i.sentence_id, ent.start, ent.end))

In [13]:
error_report(df, spans_dev_gold, data)

Sentence: CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .
  FN: LEICESTERSHIRE
Sentence: West Indian all-rounder Phil Simmons took four for 38 on Friday as Leicestershire beat Somerset by an innings and 39 runs in two days to take over at the head of the county championship .
  FP: four
  FP: 39
  FP: two days
  FP: 38
  FP: Friday
  FN: Somerset
Sentence: After bowling Somerset out for 83 on the opening morning at Grace Road , Leicestershire extended their first innings by 94 runs before being bowled out for 296 with England discard Andy Caddick taking three for 83 .
  FP: 94
  FP: 296
  FP: three
  FP: 83
  FP: the opening morning
  FP: first
  FP: 83
Sentence: Trailing by 213 , Somerset got a solid start to their second innings before Simmons stepped in to bundle them out for 174 .
  FP: 213
  FP: 174
  FP: second
Sentence: Essex , however , look certain to regain their top spot after Nasser Hussain and Peter Such gave them a firm grip on their match against Yorksh

most of the FP are number related entities (dates, time, numbers etc.)

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to post-process the output produced by spaCy. To filter out specific labels it is useful to know the named entity label scheme, which can be found in the [model's documentation](https://spacy.io/models/en#en_core_web_sm). You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [47]:
df = df_dev 
spans_dev_gold = set(gold_spans(df))
data = set()
for i in df.drop_duplicates('sentence_id').itertuples():
    doc = nlp(i.sentence)
    for ent in doc.ents:
        if ent.label_ not in ["CARDINAL", "DATE", "MONEY", "ORDINAL", "PERCENT", "QUANTITY", "TIME"]:
            data.add((i.sentence_id, ent.start, ent.end))

In [48]:
evaluation_report(spans_dev_gold, data)

precision: 0.8334364048649763 , recall:  0.683285448707115 , f1:  0.7509286775631501


In [49]:
error_report(df, spans_dev_gold, data)

Sentence: CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .
  FN: LEICESTERSHIRE
Sentence: West Indian all-rounder Phil Simmons took four for 38 on Friday as Leicestershire beat Somerset by an innings and 39 runs in two days to take over at the head of the county championship .
  FN: Somerset
Sentence: Essex , however , look certain to regain their top spot after Nasser Hussain and Peter Such gave them a firm grip on their match against Yorkshire at Headingley .
  FN: Headingley
Sentence: Hussain , considered surplus to England 's one-day requirements , struck 158 , his first championship century of the season , as Essex reached 372 and took a first innings lead of 82 .
  FN: Hussain
Sentence: By the close Yorkshire had turned that into a 37-run advantage but off-spinner Such had scuttled their hopes , taking four for 24 in 48 balls and leaving them hanging on 119 for five and praying for rain .
  FN: Such
Sentence: At the Oval , Surrey captain Chris Lewis , another man

  FN: Fleming
  FN: Gillespie
Sentence: 6-0-26-0 , Bevan 4-0-18-0 .
  FP: Bevan 4-0-18-0 .
  FN: Bevan
Sentence: CRICKET - AUSTRALIA 228-9 IN 50 OVERS V SRI LANKA .
  FP: V SRI LANKA
  FN: AUSTRALIA
  FN: SRI LANKA
Sentence: Australia scored 228 for nine wickets in their 50 overs against Sri Lanka in the third day-night limited overs match of the Singer World Series tournament on Friday .
  FP: the Singer World Series
  FN: Singer World Series
Sentence: CRICKET - AUSTRALIA WIN TOSS AND CHOOSE TO BAT .
  FN: AUSTRALIA
Sentence: overs cricket match in the Singer world series tournament on
  FP: overs cricket match
  FN: Singer
Sentence: Australia - Ian Healy ( captain ) , Michael Bevan , Damien Flemming , Jason Gillespie , Stuart Law , Glenn McGrath , Ricky Ponting , Michael Slater , Darren Lehmann , Mark Waugh , Steve Waugh .
  FN: Ian Healy
Sentence: Sri Lanka - Arjuna Ranatunga ( captain ) , Sanath Jayasuriya ,
  FP: Ranatunga
  FP: Sri Lanka - Arjuna
  FN: Sri Lanka
  FN: Arjuna Rana

  FP: line-up
Sentence: The Brazilian 's 79th-minute effort was enough to earn Real a point from a scrappy 1-1 draw at fellow title contenders Deportivo Coruna .
  FN: Real
Sentence: Deportivo started strongly , taking the lead midway through the first half when former Auxerre playmaker Corentine Martins headed home a corner after a flick-on by Brazilian-born Spanish international midfielder Donato .
  FN: Brazilian-born
  FN: Deportivo
Sentence: Real looked to be in deep trouble shortly after the break when Luis Milla was sent off for committing two bookable offences in as many minutes .
  FN: Real
Sentence: Shortly afterwards Roberto Carlos found space in the home defence and equalised for Real with a shot that was deflected past despairing Deportivo ' keeper Jacques Songo'o .
  FN: Real
  FN: Jacques Songo'o
Sentence: In a frantic final five minutes there were chances at both ends , and Donato , who had earlier been booked , was sent off for protesting about the incursion of Real pl

Sentence: Shrewsbury 0 Brentford3
  FN: Brentford3
  FN: Shrewsbury
Sentence: Postponed : Walsall v Wrexham
  FN: Walsall
  FN: Wrexham
Sentence: Brighton 1 Scunthorpe 1
  FN: Scunthorpe
Sentence: Cambridge United 0 Cardiff 2
  FP: Cambridge
  FN: Cardiff
  FN: Cambridge United
Sentence: Colchester 1 Hereford 1
  FP: Colchester 1 Hereford 1
  FN: Hereford
  FN: Colchester
Sentence: Doncaster 3 Darlington 2
  FN: Darlington
  FN: Doncaster
Sentence: Fulham 1 Carlisle 0
  FN: Carlisle
Sentence: Hull 0 Barnet 0
  FP: Hull 0 Barnet
  FN: Hull
  FN: Barnet
Sentence: Leyton Orient 2 Hartlepool 0
  FP: Leyton Orient 2
  FN: Leyton Orient
  FN: Hartlepool
Sentence: Mansfield 0 Rochdale 0
  FN: Rochdale
Sentence: Scarborough 1 Northampton 1
  FN: Scarborough
  FN: Northampton
Sentence: Torquay 2 Exeter 0
  FN: Torquay
  FN: Exeter
Sentence: Wigan 4 Chester 2
  FN: Chester
Sentence: CRICKET - ENGLISH COUNTY CHAMPIONSHIP SCORES .
  FP: CRICKET - ENGLISH COUNTY CHAMPIONSHIP SCORES
  FN: ENGLISH CO

  FP: Roberto Carlos 79th
  FN: division
  FN: Deportivo Coruna
  FN: Real Madrid
  FN: Roberto Carlos
Sentence: SOCCER - RESULT IN SPANISH FIRST DIVISION .
  FP: SPANISH FIRST DIVISION
  FN: SPANISH
Sentence: MADRID 1996-08-31
  FN: MADRID
Sentence: Spanish first division on Saturday :
  FN: division
Sentence: Deportivo Coruna 1 Real Madrid 1
  FN: Deportivo Coruna
  FN: Real Madrid
Sentence: SOCCER - BELGIUM BEAT TURKEY 2-1 IN WORLD CUP QUALIFIER .
  FP: WORLD CUP QUALIFIER
  FN: TURKEY
  FN: WORLD CUP
  FN: BELGIUM
Sentence: Turkey - Sergen Yalcin ( 61st )
  FN: Sergen Yalcin
Sentence: SOCCER - AUSTRIA DRAW 0-0 WITH SCOTLAND IN WORLD CUP QUALIFIER .
  FN: WORLD CUP
  FN: SCOTLAND
  FN: AUSTRIA
Sentence: BOXING - KNOCK-OUT SPECIALIST MILLER DEFENDS TITLE .
  FN: MILLER
Sentence: DUBLIN 1996-08-31
  FN: DUBLIN
Sentence: Heath did score with two brusing lefts to Miller 's head in the third round but failed to put his opponent under any real pressure .
  FN: Heath
Sentence: SOCCER - DUT

Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the *predicted* start and end positions for each token span, rather than the gold positions. As the `label` of each span, you can use the special value `--NME--`.

In [14]:
import pandas as pd

df = df_dev 
spans_dev_gold = set(gold_spans(df))
data = set()
for i in df.drop_duplicates('sentence_id').itertuples():
    doc = nlp(i.sentence)
    for ent in doc.ents:
        if ent.label_ not in ["CARDINAL", "DATE", "MONEY", "ORDINAL", "PERCENT", "QUANTITY", "TIME"]:
            data.add((i.sentence_id, i.sentence, ent.start, ent.end, "--NME--"))

df_dev_2 = pd.DataFrame(list(data), columns = ["sentence_id", "sentence", "beg", "end", "label"])

## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [55]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    # TODO: Replace the next line with your own code
    for i in df.itertuples():
        yield (i.sentence_id, i.beg, i.end, i.label)
        
spans_dev_gold = set(gold_mentions(df_dev))

A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

**Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.**

In [69]:
data = set()
for i ,row in df_dev_2.iterrows():
    words = row.sentence.split()
    data.add((row.sentence_id, row.beg, row.end, "_".join(words[row.beg:row.end])))

In [70]:
evaluation_report(spans_dev_gold, data)

precision: 0.3141620284477427 , recall:  0.2575629541997634 , f1:  0.28306092124814264


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on) as mentions of the entities (pages) that they link to. This allows us to harvest long lists of mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [71]:
with bz2.open('kb.tsv.bz2', 'rt', encoding = 'cp850') as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [72]:
df_kb.loc[df_kb.mention == 'Sweden']

,mention,entity,prob
17436,Sweden,Sweden,0.985768
17437,Sweden,Sweden_national_football_team,0.014173
17438,Sweden,Sweden_men's_national_ice_hockey_team,0.000059


As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing come first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [79]:
dat = set()
for i in data:
    df_filt = df_kb.loc[df_kb.mention == i[3]]
    if df_filt.shape[0] != 0: 
         dat.add((i[0],i[1],i[2],df_filt.iloc[0,1]))
    else:
         dat.add((i[0],i[1],i[2], "--NME--"))

In [80]:
evaluation_report(spans_dev_gold, dat)

precision: 0.4850546279117708 , recall:  0.3976677370288998 , f1:  0.43703566121842496


## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier that is trained to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [84]:
with bz2.open('contexts.tsv.bz2') as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

In [85]:
df_contexts.head()

,mention,entity,context
0,1970,UEFA_Champions_League,Cup twice the first in @ and the second in 1983
1,1970,FIFA_World_Cup,America 1975 and during the @ and 1978 World C...
2,1990 World Cup,1990_FIFA_World_Cup,Manolo represented Spain at the @
3,1990 World Cup,1990_FIFA_World_Cup,Hašek represented Czechoslovakia at the @ and ...
4,1990 World Cup,1990_FIFA_World_Cup,renovations in 1989 for the @ The present capa...


Note that, in each context, the position of the mention is indicated by the `@` symbol.

From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [86]:
df_contexts.context[df_contexts.mention == 'Lincoln']

41465    Nebraska Concealed Handgun Permit In @ municip...
41466    Lazlo restaurants are located in @ and Omaha C...
41467    California Washington Overland Park Kansas @ N...
41468    City Missouri Omaha Nebraska and @ Nebraska It...
41469    by Sandhills Publishing Company in @ Nebraska USA
                               ...                        
41609                                      @ Leyton Orient
41610                    English division three Swansea @ 
41611    league membership narrowly edging out @ on goa...
41612                                          @ Cambridge
41613                                                   @ 
Name: context, Length: 149, dtype: object

Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [157]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

def predict_entity(mention, sentence):
    filtered_context = df_contexts[df_contexts.mention == mention]
    X = filtered_context.context
    Y = filtered_context.entity
    priors = list(df_kb.loc[df_kb.mention == mention].sort_values("entity").prob)
    
    if filtered_context.shape[0] == 0 and len(priors) == 0:
        return "--NME--"
    elif filtered_context.shape[0] == 0 and len(priors) != 0:
        return list(df_kb.loc[df_kb.mention == mention].entity)[0]
    else: 
        model = Pipeline([('vect', CountVectorizer()), 
                            ('MNB', MultinomialNB(class_prior = priors))])

        model.fit(X = X, y = Y)

        entity = model.predict([sentence])[0]
        return entity


In [158]:
data = set()
for i ,row in df_dev_2.iterrows():
    words = row.sentence.split()
    mention = "_".join(words[row.beg:row.end])
    entity = predict_entity(mention, row.sentence)
    data.add((row.sentence_id, row.beg, row.end, entity))

In [159]:
evaluation_report(spans_dev_gold, data)

precision: 0.4955679241393527 , recall:  0.4062869697481832 , f1:  0.4465081723625557


You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1.

## Reflection questions

The following reflection questions are questions that you could be asked in the oral exam. Try to answer each of them in the form of a short text and enter it in the cell below. You will get feedback on your answers from your lab assistant.

**RQ 5.1:** In Problem&nbsp;3, you did an error analysis on the task of recognizing text spans mentioning named entities. Summarize your results. Pick one type of error that you observed. How could you improve the model&rsquo;s performance on this type of error? What resources (such as domain knowledge, data, compute) would you need to implement this improvement?

We can see that geographical names are not predicted. Well we used Spacy for predictions, it looks like spacy is not trained on geographical names and this could be solved by training it. Domain knowledge as a list of geographical names would be necessary.

**RQ 5.2:** Thinking back about Problem&nbsp;6, explain what the word *context* refers to in the task addressed there, and how context can help to disambiguate between different entities. Suggest other types of context that you could use for disambiguation.

context is a sentence where the entity is contained. What this context does is enabling us to decide what entity matches best the sentence we are predicting on. We also base this context consideration on the mention this sentence is related to. As a context we could also use POS tagging. 



**RQ 5.3:** One type of entity mentions that we did not cover explicitly in this lab are pronouns. As an example, consider the sentence pair *Ruth Bader Ginsburg was an American jurist*. *She served as an associate justice of the Supreme Court from 1993 until her death in 2020*. What facts would you want to extract from this sentence pair? How do pronouns make fact extraction hard?

- Ruth Bader Ginsburg - American jurist
- Ruth Bader Ginsburg - served as an associate justice of the Supreme Court from 1993
- Ruth Bader Ginsburg - death in 2020

because in seperate sentences we dont know to whom pronouns are referring. when the name is stated expicitly we are sure that what is being said is about the subject and otherwise when we have pronouns its more diffcult to identify the subject


**This was the last lab in the Text Mining course. Congratulations! 🥳**